In [1]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('sap_storing_data_hu_project.csv', low_memory = False)  

# Remove duplicates, date time convert, hersteltijd created, dropped nans

In [3]:
df = df.drop_duplicates(subset=['#stm_sap_meldnr'])
df = df.reset_index(drop=True)

df['stm_aanntpl_tijd']= pd.to_datetime(df['stm_aanntpl_tijd'])
df['stm_fh_ddt']= pd.to_datetime(df['stm_fh_ddt'])

# Calculate the true reparation time
df['stm_hers_tijd'] = (df['stm_fh_ddt'].dt.hour * 60 + df['stm_fh_ddt'].dt.minute) - (df['stm_aanntpl_tijd'].dt.hour * 60 + df['stm_aanntpl_tijd'].dt.minute)

df = df.dropna(subset=['stm_hers_tijd','stm_prioriteit' ]) 
df = df.reset_index(drop=True)

# Linear regression prioriteit

In [4]:
# df_og = df[['#stm_sap_meldnr', 'stm_fh_duur', 'stm_oorz_groep','stm_prioriteit', 'stm_geo_mld']]

# df_og = df_og.drop_duplicates(subset=['#stm_sap_meldnr'])

# df_og = df_og.reset_index(drop=True)

# df_og = df_og.dropna(subset=['stm_fh_duur','stm_prioriteit' ]) 

# df_og = df_og.reset_index(drop=True)

# for c, i in enumerate(df_og['stm_geo_mld']):
#     try:
#         df_og.at[c, 'stm_geo_mld'] = int(float(i))
#     except:
#         df_og.drop([c], inplace=True)
        
# df_og = df_og.reset_index(drop=True)

In [5]:
# # decide what the quantiles are
# Q1 = df_og['stm_fh_duur'].quantile(0.25)
# Q3 = df_og['stm_fh_duur'].quantile(0.75)
# IQR = Q3 - Q1
# print(IQR)

# # delete the outliers
# only_duur = df_og['stm_fh_duur']
# df_og['strong_outlier'] = ((only_duur < (Q1 - 1.5 * IQR)) | (only_duur > (Q3 + 1.5 * IQR))) #create temporary column
# df_og = df_og.drop(df_og[df_og.strong_outlier == True].index)

# df_og =  df_og.drop('strong_outlier', 1)


In [6]:
# # DummiesX = pd.get_dummies(data=df_og[['stm_geo_mld']])#, 'stm_prioriteit', 'stm_oorz_groep']])
# # DummiesX = pd.concat([DummiesX,df_og['stm_prioriteit']], axis = 1)
# # DummiesX.head()
# X = df_og[['stm_prioriteit']]
# y = df_og['stm_fh_duur']

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [8]:
# linreg = LinearRegression()
# linreg.fit(X_train, y_train)

# # print(linreg.intercept_)
# # print(linreg.coef_)

# y_pred = linreg.predict(X_test)

# np.sqrt(metrics.mean_squared_error(y_test,y_pred))
# r2_score(y_test,y_pred)

In [9]:
# y_train.mean()

In [10]:
# np.sqrt(metrics.mean_squared_error(y_test,y_pred))

# Decision Tree regression prioriteit en oorzaak groep

In [11]:
from sklearn.tree import DecisionTreeRegressor

In [12]:
df_pog = df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_geo_mld']]

## Drop duplicates, drop nan's

In [13]:
# Drop alle duplicates mbv #stm_sap_meldnr
df_pog = df_pog.drop_duplicates(subset=['#stm_sap_meldnr'])

df_pog = df_pog.reset_index(drop=True)

# Drop rij als er maar 1 nan values tussen deze kolommen zit.
df_pog = df_pog.dropna(subset=['stm_hers_tijd','stm_prioriteit', 'stm_oorz_groep']) 

df_pog = df_pog.reset_index(drop=True)

## Drop outliers

In [14]:
df_pog = df_pog[(df_pog['stm_hers_tijd'] >= 4) & (df_pog['stm_hers_tijd'] <= 480)]

df_pog = df_pog.reset_index(drop=True)

## Get dummies for oorzaakgroep and concat with prioriteit

In [15]:
DummiesX = pd.get_dummies(data=df_pog[['stm_oorz_groep']])#, 'stm_prioriteit', 'stm_oorz_groep']])
DummiesX = pd.concat([DummiesX,df_pog['stm_prioriteit']], axis = 1)
DummiesX.head()
y = df_pog['stm_hers_tijd']

## Train data

In [16]:
X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)

In [17]:
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_2 = DecisionTreeRegressor(max_depth=7)
regr_1.fit(X_train,y_train)
regr_2.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=7)

## Predict data

In [18]:
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

In [19]:
np.sqrt(metrics.mean_squared_error(y_test,y_1))

76.57518088078423

In [20]:
np.sqrt(metrics.mean_squared_error(y_test,y_2))

76.54826025960315

In [21]:
r2_score(y_test, y_1)

0.026215099353142812

In [22]:
r2_score(y_test, y_2)

0.026899662839435123

# Decision Tree regression prioriteit en oorzaak groep en oorzaakcode

In [23]:
df_Dpogc = df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_oorz_code']]

In [24]:
# Drop alle duplicates mbv #stm_sap_meldnr
df_Dpogc = df_Dpogc.drop_duplicates(subset=['#stm_sap_meldnr'])

df_Dpogc = df_Dpogc.reset_index(drop=True)

# Drop rij als er maar 1 nan values tussen deze kolommen zit.
df_Dpogc = df_Dpogc.dropna(subset=['stm_hers_tijd','stm_prioriteit', 'stm_oorz_groep', 'stm_oorz_code']) 

df_Dpogc = df_Dpogc.reset_index(drop=True)

In [25]:
df_Dpogc

#stm_sap_meldnr  stm_hers_tijd stm_oorz_groep  stm_prioriteit  \
0              50053211          540.0        ONR-RIB             9.0   
1              50053213          806.0       ONR-DERD             9.0   
2              50053214         1040.0       ONR-DERD             9.0   
3              50053215         1356.0       ONR-DERD             9.0   
4              50053218          690.0        TECHONV             9.0   
...                 ...            ...            ...             ...   
445899         99003503          179.0        TECHONV             4.0   
445900         99003504            3.0        TECHONV             5.0   
445901         99003505           24.0        TECHONV             5.0   
445902         99003506           36.0        TECHONV             2.0   
445903         99999999           29.0        ONR-RIB             2.0   

        stm_oorz_code  
0               133.0  
1               143.0  
2               142.0  
3               142.0  
4               218.0  
...               ...  
445899          221.0  
445900          298.0  
445901          215.0  
445902          218.0  
445903          135.0  

[445904 rows x 5 columns]

In [26]:
df_Dpogc = df_Dpogc[(df_Dpogc['stm_hers_tijd'] >= 4) & (df_Dpogc['stm_hers_tijd'] <= 480)]
# TODO: waarom zoveel weggegooid
df_Dpogc = df_Dpogc.reset_index(drop=True)

In [47]:
df_Dpogc['stm_hers_tijd'].mean()

67.70978782451179

In [48]:
df_Dpogc.describe()

#stm_sap_meldnr  stm_hers_tijd  stm_prioriteit  stm_oorz_code
count     3.444790e+05  344479.000000   344479.000000  344479.000000
mean      7.987887e+07      67.709788        3.715713     200.765814
std       3.414743e+06      77.937001        1.380167      44.707157
min       5.005326e+07       4.000000        1.000000      33.000000
25%       8.045409e+07      19.000000        2.000000     151.000000
50%       8.059458e+07      40.000000        4.000000     215.000000
75%       8.073855e+07      83.000000        5.000000     221.000000
max       1.000000e+08     480.000000        9.000000     999.000000

In [27]:
DummiesX = pd.get_dummies(data=df_Dpogc[['stm_oorz_groep', 'stm_oorz_code']])
DummiesX = pd.concat([DummiesX,df_Dpogc['stm_prioriteit']], axis = 1)
DummiesX.head()
y = df_Dpogc['stm_hers_tijd']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)

In [29]:
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_2 = DecisionTreeRegressor(max_depth=7)
regr_1.fit(X_train,y_train)
regr_2.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=7)

In [30]:
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

In [31]:
np.sqrt(metrics.mean_squared_error(y_test,y_1))

75.91898445279033

In [32]:
np.sqrt(metrics.mean_squared_error(y_test,y_2))

75.0833719924327

In [33]:
r2_score(y_test, y_1)

0.042832919818925896

In [34]:
r2_score(y_test, y_2)

0.06378733794207447

# Knn regressor

In [35]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)

In [36]:
df_Kpog = df[['#stm_sap_meldnr', 'stm_hers_tijd', 'stm_oorz_groep','stm_prioriteit', 'stm_oorz_code']]

In [37]:
df_Kpog = df_Kpog.drop_duplicates(subset=['#stm_sap_meldnr'])

df_Kpog = df_Kpog.reset_index(drop=True)

df_Kpog = df_Kpog.dropna(subset=['stm_hers_tijd', 'stm_oorz_code', 'stm_prioriteit', 'stm_oorz_groep']) 

df_Kpog = df_Kpog.reset_index(drop=True)

In [38]:
# decide what the quantiles are
Q1 = df_Kpog['stm_hers_tijd'].quantile(0.25)
Q3 = df_Kpog['stm_hers_tijd'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

df_Kpog = df_Kpog[(df_Kpog['stm_hers_tijd'] >= 4) & (df_Kpog['stm_hers_tijd'] <= 480)]

df_Kpog = df_Kpog.reset_index(drop=True)

73.0


In [39]:
DummiesX = pd.get_dummies(data=df_Kpog[['stm_oorz_code', 'stm_oorz_groep']])#, 'stm_prioriteit', 'stm_oorz_groep']])
DummiesX = pd.concat([DummiesX,df_Kpog['stm_prioriteit']], axis = 1)
DummiesX.head()
# DummiesX = df_Kpog[['stm_prioriteit']]
y = df_Kpog['stm_hers_tijd']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(DummiesX, y, random_state=2)

In [41]:
neigh.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=2)

In [42]:
y_pred = neigh.predict(X_test)

In [43]:
np.sqrt(metrics.mean_squared_error(y_test,y_pred))

90.49498490375626

In [44]:
r2_score(y_test, y_pred)

-0.3599908840500332